In [1]:
import io, os, sys, time
from zipfile import ZipFile
import zeep
import xmltodict, json, csv
import psycopg2
from configparser import ConfigParser
from datetime import datetime
import pytz
#import ctypes
#from ctypes.wintypes import MAX_PATH
import socket
from Indicadores import Indicadores
import html

In [4]:
    def html_unescape(variável, valor):
        variável = html.unescape(valor)
    
    def recorre_sobre_todo_json(d, path=None, funcion = html_unescape):
        if not path: path = []
        for k, v in d.items():
            if isinstance(v, dict) or isinstance(v, OrderedDict):
                #print("Getting: ", k)
                recorre_sobre_todo_json(v, path + [k])
            else:
                if isinstance(v, list):
                    num = 0
                    for item in v:
                        #print("Unfolding List: ", v)
                        recorre_sobre_todo_json(item, path + [k] + [num])
                        num +=1
                else:
                    if not v == None:
                        funcion(d[k], v)
                        #print("{2} - {0} : {1}".format(k, html.unescape(v), path))
                    else:
                        #print("{2} - {0} : {1}".format(k, v, path))
                        pass

In [5]:
    def get_xml_from_zip(zip_lattes):
        if zip_lattes == None:
            return False
        try:
            with ZipFile(io.BytesIO(zip_lattes)) as myzip:
                with myzip.open(myzip.namelist()[0]) as myfile:
                    print('Unzipping file and gettinf xml and json')
                    xml = myfile.read()
                
                #transforma o Lattes compactado em um dicionário JSON
                json_lattes = xmltodict.parse(xml)
                #retira, de cada JSON, caracteres especiais web
                recorre_sobre_todo_json(json_lattes)

                #Decoding XML - must be done after JSON transform.
                xml = xml.decode('iso-8859-1').replace('encoding="ISO-8859-1" ', '')
            print("XML & JSON atualizado a partir do Lattes compactado.")
            return True
        except:
            print("Error in File - Could not retrieve zipped information.")
            return False

In [9]:
    def read_zip_from_disk(id):
        filename = "Lattes_"+ id + ".zip"
        file_path = os.path.join('C:/Downloads/', filename)
        with open(file_path , 'rb') as f:
            zip_lattes = f.read()
        print("Arquivo compactado lido no disco.")
        return zip_lattes


In [10]:
a = read_zip_from_disk('7281587998425548')

Arquivo compactado lido no disco.


In [11]:
print(a)

b'PK\x03\x04\x14\x00\x08\x00\x08\x00\xcdLDT\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x14\x00\x00\x007281587998425548.xml\xed}\xcbr\xe3\xc8\x95\xe8\xafd\xd0\x11sU\x11\xa4$>\xc4\x92\xecR9 \x12T\xc1&\t\x16A\xaa\xcb\xbdQ@$D\xa1\x1b\x04\xd8x\xa8Z\x1d\xf3\x1d\xf31\xb3\xbb\xf73\xee\xca5\xbd\xe8(\xc7\xadU\xcfl\xbc\xba\xe7\x9c\xcc\xc4\x83\x04(\x89%\xa9$Y\x9e\xb1K\x04\x12\x99y\xde\x8f<\x99\xf9\xe6\xcf?\xcf\x1dvi\xf9\x81\xed\xb9\x87\xa5\xea\xf6n\x89Y\xee\xc4\x9b\xda\xee\xec\xb0\xa4\x19ze\x7f\x7f\xef\xa0R-\xb1 4\xdd\xa9\xe9x\xaeuXr\xbd\xd2\x9f\xdf\xbei\x8d\x87C\xad5\xee\xea\x95\x13m\xa4\xa8\xcc\xd0\x8c\x91\xdaS*\xfaP;V{\x95\x0f\xbd\xeea\xa9\xab\x8cF\xaaq\xaaw:]\xad\xaf\x96X\x7f\xdcS\x87zEk\xab\xfd\x91\xd6\xd1ZJ[\x1f\x1e\x96^\xd7\xf6\xab{\xfb\xaf\x0f\x0e\xf6\x1b\xb5\xbd\xbd\xc6~\x89\xb5\x95\x91RQFc\xa5\xab}\xaf\xb4\x14\xfd\xb0\xb4[\xdf\xad\xd5\xe0\xffK\xec\x9d>\\zW\xad\xd6\x9b{\xf5\xd2\xdb7m\xe8\xcf\xa8\x1c\xabCE3X_\xef\xa9\x95\x96\xde\x1bt\xd5\x114R\x9c3\xcb\x0f=6\xb5X\xcb\x9c/\xbc\x80Y\xcc

In [22]:
from Lattes import Lattes
lattes = Lattes()
lattes.id = '7281587998425548'
lattes.read_zip_from_disk()


Arquivo compactado lido no disco.


In [23]:
lattes.get_xml_from_zip()

Unzipping file and gettinf xml and json
XML & JSON atualizado a partir do Lattes compactado.


True

In [21]:
print(lattes.json_lattes)

OrderedDict([('CURRICULO-VITAE', OrderedDict([('@SISTEMA-ORIGEM-XML', 'LATTES_OFFLINE'), ('@NUMERO-IDENTIFICADOR', '7281587998425548'), ('@DATA-ATUALIZACAO', '03022022'), ('@HORA-ATUALIZACAO', '113653'), ('DADOS-GERAIS', OrderedDict([('@NOME-COMPLETO', 'Alberto de Campos e Silva'), ('@NOME-EM-CITACOES-BIBLIOGRAFICAS', 'Campos Silva, A'), ('@NACIONALIDADE', 'B'), ('@PAIS-DE-NASCIMENTO', 'Brasil'), ('@UF-NASCIMENTO', 'GO'), ('@CIDADE-NASCIMENTO', 'Goiânia'), ('@PERMISSAO-DE-DIVULGACAO', 'NAO'), ('@DATA-FALECIMENTO', ''), ('@SIGLA-PAIS-NACIONALIDADE', 'BRA'), ('@PAIS-DE-NACIONALIDADE', 'Brasil'), ('@ORCID-ID', 'https://orcid.org/0000-0001-6506-1988'), ('RESUMO-CV', OrderedDict([('@TEXTO-RESUMO-CV-RH', 'Sou casado com uma mulher maravilhosa e tenho três lindos. Atualmente, sou Analista de C&T no CNPq, lotado na COCEX/CGCEX/DEHS, analisando principalmente processos relacionados à Física e à Matemática. Tenho 46 anos, sou formado em física pela UnB, tendo dado aulas (de física) para o ensino

In [15]:
from collections import OrderedDict

In [19]:
jhg = recorre_sobre_todo_json(lattes.json_lattes)

In [20]:
print(jhg)

None
